In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

In [11]:
data = pd.read_csv('./IMDB Dataset.csv')

data['sentiment'].value_counts()

data.drop_duplicates(inplace=True)


In [ ]:
def clean_review(review):
    str = " ".join(word for word in review.split() if word not in stopwords.words('english'))
    return str

review = ' '.join(word for word in data [])